# Inputs

In [1]:
import os
import pandas as pd
import yaml

In [2]:
DATASET_PARAMS = yaml.load(open("configs/dataset.yaml"), Loader=yaml.SafeLoader)['DATASET']
PREPROCESSING_PARAMS = yaml.load(open("configs/preprocessing.yaml"), Loader=yaml.SafeLoader)['PREPROCESSING']
EFFICIENTCAPSNET_PARAMS = yaml.load(open("configs/efficientcapsnet.yaml"), Loader=yaml.SafeLoader)['CAPSNET']
TRAINING_PARAMS = yaml.load(open("configs/training.yaml"), Loader=yaml.SafeLoader)

In [3]:
categories =  DATASET_PARAMS['CATEGORIES']

BASE_DIR = os.getcwd()
DATASET_DIR = os.path.join(BASE_DIR, DATASET_PARAMS['DATA_PATH'])

TRAIN_DIR = os.path.join(DATASET_DIR, "train")
VAL_DIR = os.path.join(DATASET_DIR, "val")
TEST_DIR = os.path.join(DATASET_DIR, "test")

In [4]:
import torch

In [5]:
torch.__version__

'2.7.0+cu118'

In [6]:
from src.utils import get_device
DEVICE = get_device()

# Dataset

In [7]:
from src.utils import generate_filenames_df

In [8]:
train_filenames_df = generate_filenames_df(TRAIN_DIR, categories)
val_filenames_df = generate_filenames_df(VAL_DIR, categories)

## Pre Processor

In [9]:
from src.preprocessing import preprocess

target_input_size = tuple(PREPROCESSING_PARAMS['INPUT_SIZE'])
train_transform = preprocess(
    target_input_size=target_input_size,
    rotation_range=PREPROCESSING_PARAMS['ROTATION_RANGE'],
    width_shift_range=PREPROCESSING_PARAMS['WIDTH_SHIFT_RANGE'],
    height_shift_range=PREPROCESSING_PARAMS['HEIGHT_SHIFT_RANGE'],
    brightness_range=PREPROCESSING_PARAMS['BRIGHTNESS_RANGE'],
    zoom_range=PREPROCESSING_PARAMS['ZOOM_RANGE'],
    horizontal_flip=PREPROCESSING_PARAMS['HORIZONTAL_FLIP'],
    vertical_flip=PREPROCESSING_PARAMS['VERTICAL_FLIP'],
    channel_shift_range=PREPROCESSING_PARAMS['CHANNEL_SHIFT_RANGE'],
    fill_mode=PREPROCESSING_PARAMS['FILL_MODE'],
    drop_out=PREPROCESSING_PARAMS['DROP_OUT'],
    shear_range=PREPROCESSING_PARAMS['SHEAR_RANGE']
    )

val_transform = preprocess(target_input_size=target_input_size) # only rescaling

In [10]:
from src.dataset import Dataset

dataset = Dataset(dataframe = train_filenames_df,
    transform=train_transform
)

## Train loader

In [11]:
from src.loader import Loader
train_loader = Loader(train_filenames_df, 
                     batch_size=DATASET_PARAMS['BATCH_SIZE'], 
                     num_workers=EFFICIENTCAPSNET_PARAMS['NUM_WORKERS'], 
                     transform=train_transform, 
                     shuffle=True)

val_loader = Loader(val_filenames_df, 
                    batch_size=DATASET_PARAMS['BATCH_SIZE'], 
                    num_workers=EFFICIENTCAPSNET_PARAMS['NUM_WORKERS'], 
                    transform=val_transform, 
                    shuffle=False)


# Model

In [12]:
from src.model import EfficientCapsNet

model = EfficientCapsNet(input_size=(EFFICIENTCAPSNET_PARAMS['INPUT_SIZE']))



# Training

## Metrics

In [13]:
# Loss
from src.loss import MarginLoss

margin_loss = MarginLoss()

# Optimizer
from torch.optim import Adam

optimizer = Adam(model.parameters(), lr=TRAINING_PARAMS['LEARNING_RATE'])

# Metrics
from src.metrics import (
    MatthewsCorrelationCoefficient,
    CustomPrecision,
    CustomRecall,
    CustomAUC,
    CustomSpecificity
)
metrics = {
    "mcc": MatthewsCorrelationCoefficient(num_classes=EFFICIENTCAPSNET_PARAMS['NUM_CLASSES']),
    "precision": CustomPrecision(num_classes=EFFICIENTCAPSNET_PARAMS['NUM_CLASSES'], average=TRAINING_PARAMS['AVERAGE']),
    "recall": CustomRecall(num_classes=EFFICIENTCAPSNET_PARAMS['NUM_CLASSES'], average=TRAINING_PARAMS['AVERAGE']),
    "auc": CustomAUC(num_classes=EFFICIENTCAPSNET_PARAMS['NUM_CLASSES'], average=TRAINING_PARAMS['AVERAGE']),
    "specificity": CustomSpecificity(num_classes=EFFICIENTCAPSNET_PARAMS['NUM_CLASSES'], average=TRAINING_PARAMS['AVERAGE'])
}

In [14]:
dataset[0][1]

'1'

In [15]:
from src.train import train
from src.utils import get_device

history = train(model=model, 
    train_loader=train_loader, 
    val_loader=val_loader, 
    criterion=margin_loss, 
    optimizer=optimizer, 
    num_epochs=TRAINING_PARAMS['NUM_EPOCHS'], 
    device=get_device(),
    metrics=metrics,
    print_every=TRAINING_PARAMS['PRINT_EVERY'],
    save_patience=TRAINING_PARAMS['SAVE_PATIENCE'],
    save_path=TRAINING_PARAMS['SAVE_PATH'],
    save_model=TRAINING_PARAMS['SAVE_MODEL'],
    save_metrics=TRAINING_PARAMS['SAVE_METRICS']
    )


cuda
Training...
Initial memory usage: {'cpu_memory': '588.14MB', 'gpu_memory_allocated': '0.63MB', 'gpu_memory_reserved': '2.00MB'}


  0%|          | 0/10 [00:00<?, ?it/s]


Epoch 1/10
Memory before epoch: {'cpu_memory': '588.87MB', 'gpu_memory_allocated': '0.63MB', 'gpu_memory_reserved': '2.00MB'}
torch.Size([1, 3, 299, 299])
torch.Size([1, 32, 295, 295])
torch.Size([1, 64, 293, 293])
torch.Size([1, 64, 291, 291])
torch.Size([1, 128, 145, 145])
torch.Size([18769, 16, 8])
torch.Size([18769, 10, 16])


  0%|          | 0/10 [00:01<?, ?it/s]

outputs: (tensor([[[ 1.1288e-04,  1.6251e-04, -5.3558e-04,  ..., -3.8232e-04,
           2.8454e-04,  3.1249e-04],
         [-1.3150e-04,  2.1831e-04, -1.4730e-04,  ...,  9.0156e-05,
           2.4672e-04,  8.0964e-04],
         [-4.4187e-04, -4.4852e-05, -3.3703e-04,  ...,  7.9076e-06,
          -2.7849e-07,  4.2405e-04],
         ...,
         [ 4.1881e-05,  1.3196e-05,  4.6815e-05,  ..., -3.7386e-04,
           6.3164e-04, -5.0603e-05],
         [ 6.3430e-05,  4.2062e-04,  4.4831e-04,  ...,  5.1477e-05,
           3.5667e-04,  6.8358e-04],
         [-4.8589e-04, -2.1647e-04,  3.7448e-04,  ...,  3.4327e-04,
          -9.7737e-05, -3.0493e-04]],

        [[ 1.1288e-04,  1.6251e-04, -5.3558e-04,  ..., -3.8232e-04,
           2.8454e-04,  3.1249e-04],
         [-1.3150e-04,  2.1831e-04, -1.4730e-04,  ...,  9.0156e-05,
           2.4672e-04,  8.0964e-04],
         [-4.4187e-04, -4.4852e-05, -3.3703e-04,  ...,  7.9076e-06,
          -2.7849e-07,  4.2405e-04],
         ...,
         [ 4.18

AttributeError: 'tuple' object has no attribute 'shape'